In [1]:
%load_ext autoreload
%autoreload 1

%aimport pyscisci
import random

In [2]:
Npubs = 10
Nauthors = 3

database = pyscisci.BibDatabase()

for pid in range(10):
    pubdict = {'id':pid, 'year':2000+pid,
               'title':'', 
               'authors':random.sample(list(range(Nauthors)), 2),
               'references':list(range(pid+1)), 
               'citations':list(range(pid+1, Npubs))}
    database.add_pub(pyscisci.Publication(database).from_dict(pubdict))
    
for aid in range(Nauthors):
    authordict = {'id':aid, 'fullname':'Example Author{}'.format(aid),
                 'publications':[pid for pid in database.publication_ids if aid in database.get_pub(pid).authors]}
    database.add_author(pyscisci.Author(database).from_dict(authordict))
    
    
print(database.publication_years)
print(database.start_year, database.end_year)

print("{0} publications and {1} authors".format(database.n_publications, database.n_authors))

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]
2000 2009
10 publications and 3 authors


In [3]:
for pid in range(Npubs):
    pub = database.get_pub(pid)
    print(pub.id, pub.year, pub.ck(4), pub.c10, pub.total_citations, pub.disruption_index)
    print("references: {0} | citations: {1}".format(pub.references, pub.citations))
    print(pub.cocited_publications)
    print(pub.yearly_cnorm)
    print()

0 2000 4 9 9 -1.0
references: [0] | citations: [1, 2, 3, 4, 5, 6, 7, 8, 9]
{1, 2, 3, 4, 5, 6, 7, 8, 9}


AttributeError: 'int' object has no attribute 'yearly_citations'

In [ ]:
for aid in range(Nauthors):
    author = database.get_author(aid)
    print(author.start_year, author.end_year)
    print(author.career_length, author.total_productivity, author.annual_productivity)
    print("Total citations: {0}, H-index: {1}".format(author.total_citations, author.hindex))
    print()
print(author.cummulative_yearly_citations)
print(author.author_timeline)